<a href="https://colab.research.google.com/github/yuji-sgs/Practical-Statistics-for-Data-Scientists/blob/main/Chapter1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1章：探索的データ分析

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
# import wquantiles

import seaborn as sns
import matplotlib.pylab as plt

In [28]:
dir = Path('drive/MyDrive/Colab Notebooks/技術書/データサイエンスのための統計学入門/data')

In [29]:
AIRLINE_STATS_CSV = dir / 'airline_stats.csv'
KC_TAX_CSV = dir / 'kc_tax.csv.gz'
LC_LOANS_CSV = dir / 'lc_loans.csv'
AIRPORT_DELAYS_CSV = dir / 'dfw_airline.csv'
SP500_DATA_CSV = dir / 'sp500_data.csv.gz'
SP500_SECTORS_CSV = dir / 'sp500_sectors.csv'
STATE_CSV = dir / 'state.csv'